## Part 1 - Terminal processing in CloudCompare

"C:\Program Files\CloudCompare\CloudCompare.exe" -SILENT -o -GLOBAL_SHIFT AUTO “1.laz” -COORD_TO_SF Z -SOR 6 1 -NOISE RADIUS 0.5 REL 1.0 -CSF -SCENES SLOPE -CLOTH_RESOLUTION 2.0 -MAX_ITERATION 500 -CLASS_THRESHOLD 0.5 -EXPORT_GROUND -EXPORT_OFFGROUND


"C:\Program Files\CloudCompare\CloudCompare.exe" -SILENT -o -GLOBAL_SHIFT AUTO “1_Z_TO_SF_SOR_DENOISED_ground_points.bin” -RASTERIZE -GRID_STEP 1 -VERT_DIR 2 -PROJ MIN -SF_PROJ MIN -EMPTY_FILL INTERP -OUTPUT_RASTER_Z


"C:\Program Files\CloudCompare\CloudCompare.exe" -SILENT -o -GLOBAL_SHIFT AUTO “1_Z_TO_SF_SOR_DENOISED_offground_points.bin” -RASTERIZE -GRID_STEP 1 -VERT_DIR 2 -PROJ MAX -SF_PROJ MAX -OUTPUT_RASTER_Z

## Part 2 - Canopy Height Model creation and generation of Trees and Bushes shapefiles with area attribute

In [34]:
"""
This script processes remote sensing data to calculate and analyze Canopy Height Models (CHM).
It includes functions to process CHM data, and save shapefiles for bushes and trees based on height thresholds.
"""

import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape

# Constants for file paths
CHM_FILE_PATH = 'chm.tif'
BUSHES_SHAPEFILE_PATH = 'bushes_coverage.shp'
TREES_SHAPEFILE_PATH = 'trees_coverage.shp'
#Double check the name of rasters!
OFFGROUND_RASTER_PATH = '1_Z_TO_SF_SOR_DENOISED_offground_points_RASTER_Z_2024-02-29_15h22_38_332.tif'
GROUND_RASTER_PATH = '1_Z_TO_SF_SOR_DENOISED_ground_points_RASTER_Z_2024-02-29_15h22_16_032.tif'

def calculate_chm(offground_raster_path, ground_raster_path, output_path):
    """
    Calculate the Canopy Height Model (CHM) from offground and ground raster data.
    """
    try:
        with rasterio.open(offground_raster_path) as offground_raster:
            with rasterio.open(ground_raster_path) as ground_raster:
                offground_data = offground_raster.read(1)
                ground_data = ground_raster.read(1)
                chm = offground_data - ground_data

                # Save CHM raster
                profile = offground_raster.profile
                with rasterio.open(output_path, 'w', **profile) as dst:
                    dst.write(chm, 1)
    except IOError as e:
        print(f"Error in reading/writing raster files: {e}")

def process_and_save_chm_shapefiles(chm_file_path, bushes_shapefile_path, trees_shapefile_path,
                                    bush_min_height=0.5, bush_max_height=3.0, tree_min_height=3.0):
    """
    Process CHM data and save shapefiles for bushes and trees based on specified height thresholds.
    """
    try:
        with rasterio.open(chm_file_path) as src:
            chm_data = src.read(1).astype('float32')  # Convert to float32
            transform = src.transform
            crs = src.crs

        # Create masks for bushes and trees
        bushes_mask = (chm_data >= bush_min_height) & (chm_data < bush_max_height)
        trees_mask = chm_data >= tree_min_height

        # Process and save each category
        for mask, shapefile_path in [(bushes_mask, bushes_shapefile_path), (trees_mask, trees_shapefile_path)]:
            shapes_data = shapes(chm_data, mask=mask, transform=transform)
            gdf = gpd.GeoDataFrame.from_features(
                ({'geometry': shape(geom), 'properties': {'height': value}} 
                 for geom, value in shapes_data), 
                crs=crs)
            
            # Dissolve adjacent geometries and add area attribute in hectares
            dissolved_gdf = gdf.dissolve()
            dissolved_gdf['area_ha'] = dissolved_gdf['geometry'].area / 10000

            # Save the GeoDataFrame as a shapefile
            dissolved_gdf.to_file(shapefile_path)
    except IOError as e:
        print(f"Error in processing/saving shapefiles: {e}")

# Main execution
if __name__ == "__main__":
    calculate_chm(OFFGROUND_RASTER_PATH, GROUND_RASTER_PATH, CHM_FILE_PATH)
    process_and_save_chm_shapefiles(CHM_FILE_PATH, BUSHES_SHAPEFILE_PATH, TREES_SHAPEFILE_PATH)